# Wavelets
Everyone knows the Fourier transform where (stationary) signals are represented as a sum of sinusoids, and the standard transform is only localized in frequency. [Wavelets](https://en.wikipedia.org/wiki/Wavelet) on the other hand are localized in both time and frequency. Wavelets have proven to perform well in many ML applications on non-stationary signals. The two main approaches of using wavelets are:
* Discrete wavelet transform (DWT): A signal is broken down into subbands, from which features are extracted and used as input to ML models.
* Continuous wavelet transform (CWT): A scaleogram can be constructed from the signal. The scaleogram can be compared to a spectrogram when using FFT. Scaleogram images can then be used as input to an image classification model.  

In this notebook we will look into how to create scaleograms from bird song signatures.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import io
import librosa
import warnings
warnings.filterwarnings('ignore') # get rid of librosa warnings

We could use [PyWavelets](https://pywavelets.readthedocs.io/en/latest/) for the CWT, but here we will use a modified version of [Alexander Neergaard's CWT](https://github.com/aneergaard/CWT). Specifically a parameter is added to allow cutting off the longest wavelets (lowest frequencies).

In [ ]:
import math

# Continous Wavelet Transform with Morlet wavelet 
# Original code by Alexander Neergaard, https://github.com/neergaard/CWT
# 
# Parameters:
#   data: input data
#   nv: # of voices (scales) per octave
#   sr: sampling frequency (Hz)
#   low_freq: lowest frequency (Hz) of interest (limts longest scale)
def cwt2(data, nv=10, sr=1., low_freq=0.):
    data -= np.mean(data)
    n_orig = data.size
    ds = 1 / nv
    dt = 1 / sr

    # Pad data symmetrically
    padvalue = n_orig // 2
    x = np.concatenate((np.flipud(data[0:padvalue]), data, np.flipud(data[-padvalue:])))
    n = x.size

    # Define scales
    _, _, wavscales = getDefaultScales(n_orig, ds, sr, low_freq)
    num_scales = wavscales.size

    # Frequency vector sampling the Fourier transform of the wavelet
    omega = np.arange(1, math.floor(n / 2) + 1, dtype=np.float64)
    omega *= (2 * np.pi) / n
    omega = np.concatenate((np.array([0]), omega, -omega[np.arange(math.floor((n - 1) / 2), 0, -1, dtype=int) - 1]))

    # Compute FFT of the (padded) time series
    f = np.fft.fft(x)

    # Loop through all the scales and compute wavelet Fourier transform
    psift, freq = waveft(omega, wavscales)

    # Inverse transform to obtain the wavelet coefficients.
    cwtcfs = np.fft.ifft(np.kron(np.ones([num_scales, 1]), f) * psift)
    cfs = cwtcfs[:, padvalue:padvalue + n_orig]
    freq = freq * sr

    return cfs, freq

def getDefaultScales(n, ds, sr, low_freq):
    nv = 1 / ds
    # Smallest useful scale (default 2 for Morlet)
    s0 = 2

    # Determine longest useful scale for wavelet
    max_scale = n // (np.sqrt(2) * s0)
    if max_scale <= 1:
        max_scale = n // 2
    max_scale = np.floor(nv * np.log2(max_scale)) 
    a0 = 2 ** ds
    scales = s0 * a0 ** np.arange(0, max_scale + 1)
    
    # filter out scales below low_freq
    fourier_factor = 6 / (2 * np.pi)
    frequencies = sr * fourier_factor / scales
    frequencies = frequencies[frequencies >= low_freq]
    scales = scales[0:len(frequencies)]

    return s0, ds, scales

def waveft(omega, scales):
    num_freq = omega.size
    num_scales = scales.size
    wft = np.zeros([num_scales, num_freq])

    gC = 6
    mul = 2
    for jj, scale in enumerate(scales):
        expnt = -(scale * omega - gC) ** 2 / 2 * (omega > 0)
        wft[jj, ] = mul * np.exp(expnt) * (omega > 0)

    fourier_factor = gC / (2 * np.pi)
    frequencies = fourier_factor / scales

    return wft, frequencies

## FFT vs Spectrogram vs Scaleogram
First we will take a look at the difference between the standard FFT, the spectrogram and the scaleogram.

In [ ]:
import scipy.fftpack
from scipy import signal
from scipy.signal import chirp

DB_RANGE = 100 # dynamic range to show in dB
SR = 22050
CMAP = 'magma'

def show_sigx3(d):
    fig, axes = plt.subplots(1, 3, figsize=(16,5))
    # FFT
    N, T = SR, 1./SR
    x = np.linspace(0.0, int(N*T), N)
    yf = scipy.fftpack.fft(d*np.hamming(len(d)))
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    axes[0].plot(xf,  20*np.log10(2.0/N *np.abs(yf[:N//2])))
    axes[0].set_ylim(-80,0)
    axes[0].set_title('FFT')
    # Spectrogram
    f, t, Sxx = signal.spectrogram(d, SR)
    axes[1].pcolormesh(t, f, 20*np.log10(Sxx), shading='auto', cmap=CMAP, vmax=-60, vmin=-60-DB_RANGE)
    axes[1].set_title('Spectrogram')
    # CWT
    #cs, f = calc_cwt(d)
    cs, f = cwt2(d, nv=12, sr=SR, low_freq=40)
    axes[2].imshow(20*np.log10(np.abs(cs)), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-DB_RANGE)
    axes[2].set_title('Scaleogram')
    plt.show()

## Random noise

In [ ]:
d = np.random.randn(SR)
show_sigx3(d)

The scaleogram has high temporal resolution but poor frequency resolution at small scales (high frequencies), and high frequency resolution but poor temporal resolution at large scales (low frequencies).

## Chirp

In [ ]:
t = np.linspace(0, 1, SR)
d = chirp(t, f0=50, f1=10e3, t1=1., method='hyperbolic')
show_sigx3(d)

Already we can see that scaleograms generally appears smoother than spectrograms, and with less artifacts. 

# Bird songs

Right, let's look at some bird song recordings. Will resample all recordings to mono, 22050Hz.

In [ ]:
from IPython.display import Audio
from sklearn.preprocessing import minmax_scale

TRAIN_DIR = '../input/birdsong-recognition/train_audio'
SR = 22050 # sample rate in Hz

def rd_file(fname, offset=0, duration=60):
    data, _ = librosa.load(fname, sr=SR, mono=True, offset=offset, duration=duration)
    data = minmax_scale(data-data.mean(), feature_range=(-1,1))
    return data

In [ ]:
# load a file with Black-throated Blue Warbler
d1 = rd_file(TRAIN_DIR+'/btbwar/XC139608.mp3')
Audio(d1, rate=SR)

In [ ]:
# load another file with Black-throated Green Warbler
d2 = rd_file(TRAIN_DIR+'/btnwar/XC135117.mp3')
Audio(d2, rate=SR)

* Now, make a wavelet transform of the audio recordings and plot the scaleograms for these two files. 

In [ ]:
cs1, f1 = cwt2(d1, nv=12, sr=SR, low_freq=40)
plt.figure(figsize = (16,4))
plt.imshow(20*np.log10(np.abs(cs1)), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-30)
plt.show()

In [ ]:
cs2, f2 = cwt2(d2, nv=12, sr=SR, low_freq=40)
plt.figure(figsize = (16,4))
plt.imshow(20*np.log10(np.abs(cs2)), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-30)
plt.show()

We can clearly see the calls. Let's zoom in on a single call from the two species.

In [ ]:
def plot_sigx2(d1, d2, name1='data 1', name2='data 2', cwt=True, db_range=30):
    fig, axes = plt.subplots(1, 2, figsize=(16,4))
    d = [d1, d2]
    name = [name1, name2]
    for i in range(2):
        if cwt == True:
            cs, _ = cwt2(d[i], nv=12, sr=SR, low_freq=40)
            axes[i].imshow(20*np.log10(np.abs(cs)),  cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-db_range)
        else:
            f, t, Sxx = signal.spectrogram(d[i], SR)
            axes[i].pcolormesh(t, f, 20*np.log10(Sxx), shading='auto', cmap=CMAP, vmax=-60, vmin=-60-db_range)
        axes[i].set_title(name[i])
    plt.show()

In [ ]:
plot_sigx2(d1[25000:65000], d2[45000:85000], 
           name1='Black-throated Blue Warbler', name2='Black-throated Green Warbler')

A clear difference! For comparison, let's look at the spectrograms of the same signals.

In [ ]:
plot_sigx2(d1[25000:65000], d2[45000:85000], 
           name1='Black-throated Blue Warbler', name2='Black-throated Green Warbler', cwt=False, db_range=60)

Notice that we are using a threshold function (coefficients below -30dB are set to zero) on the amplitudes of the coefficients here to remove background noise. If we include a larger dynamic range, plots look quite different:

In [ ]:
plot_sigx2(d1[25000:65000], d2[45000:85000], 
           name1='Black-throated Blue Warbler', name2='Black-throated Green Warbler', db_range=120)

In [ ]:
plot_sigx2(d1[25000:65000], d2[45000:85000], 
           name1='Black-throated Blue Warbler', name2='Black-throated Green Warbler', cwt=False, db_range=120)

One major difference between the Fourier transform and wavelet transform is that using the latter we do not have to worry about using a window function to avoid discontinuities, since wavelets are not continuous functions. We can illustrate this by loading a 10s recording, and then compare the coefficients we get by doing the transform on one 10s segment with 10 times 1s segments concatenated. Let's try a recording of the Northern Mockingbird - a bird that basically never makes the same song twice...

In [ ]:
d3 = rd_file(TRAIN_DIR+'/normoc/XC54018.mp3', offset=127, duration=10)
Audio(d3, rate=SR)

In [ ]:
cs3a, f3 = cwt2(d3, nv=12, sr=SR, low_freq=40)
cs3b=np.zeros((97,10*SR), dtype=np.complex)
for i in range(10):
    cs3b[:,i*SR:(i+1)*SR], _ = cwt2(d3[i*SR:(i+1)*SR], nv=12, sr=SR, low_freq=40)
fig, axes = plt.subplots(3, 1, figsize=(16,12))
axes[0].imshow(20*np.log10(np.abs(cs3a)), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-40)
axes[0].set_title('Scaleogram - 10s')
axes[1].imshow(20*np.log10(np.abs(cs3b)), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-40)
axes[1].set_title('Scaleogram - 10 x 1s')
diff = np.abs(np.abs(cs3a)-np.abs(cs3b))
diff[diff == 0.] = 1e-15
axes[2].imshow(20*np.log10(diff), cmap=CMAP, aspect='auto', norm=None, vmax=0, vmin=-40)
axes[2].set_title('Scaleogram - difference')
plt.show()

The scaleograms are identical - we can apply the CWT on a signal length optimized for available CPU power and memory, and split and concatenate the coefficients whichever way we want! Now, repeat same exercise with spectrogram.

In [ ]:
f, t, Sxxa = signal.spectrogram(d3, SR)
Sxxb=np.zeros((129,10*98))
for i in range(10):
    _, _, Sxxb[:,i*98:(i+1)*98] = signal.spectrogram(d3[i*SR:(i+1)*SR])
fig, axes = plt.subplots(3, 1, figsize=(16,12))
axes[0].pcolormesh(t[0:980], f, 20*np.log10(Sxxa[:,0:980]), shading='auto', cmap=CMAP, vmax=-90, vmin=-130)
axes[0].set_title('Spectrogram - 10s')
axes[1].pcolormesh(t[0:980], f, 20*np.log10(Sxxb), shading='auto', cmap=CMAP, vmax=-0, vmin=-40)
axes[1].set_title('Spectrogram - 10 x 1s')
diff = np.abs(np.abs(Sxxa[:,0:980]*3e4)-np.abs(Sxxb))
diff[diff == 0.] = 1e-15
axes[2].pcolormesh(t[0:980], f, 20*np.log10(diff), shading='auto', cmap=CMAP, vmax=-0, vmin=-40)
axes[2].set_title('Spectrogram - difference')
plt.show()

As, we can see, the spectrogram is a lot more complicated - coefficient scaling is different and we do not even get the same number of coefficients for a 10s as with 10x 1s. A summary so far:  
* Spectrogram (based on FFT): Easy to understand, hard to use
* Scaleogram (based on CWT): Hard to understand, easy to use.

So, what we need now is an automated way of extracting these bird signatures, and dump the signatures to jpeg files as input to an image classification model. There are many ways to locate signatures in recordings, and a basic, rule-based one is used below. Of course, these are easy birds, and more difficult birds (or noisy recordings) present a whole range of challenges...

# Automatic signature extraction
If we calculate the variance of each wavelet vector, we might use that as input to the extraction algorithm.

In [ ]:
# calculate variance of coefficients
def calc_var(cs, thres):
    c = 20*np.log10(np.abs(cs))
    c[c < thres] = 0.
    e = np.var(c, axis=0)
    return e / max(e)

Let's see how that works out for the two files.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16,4))
v1 = calc_var(cs1, -30)
axes[0].plot(v1)
v2 = calc_var(cs2, -30)
axes[1].plot(v2)
plt.show()

Scipy has a function find_peaks() that we could use to find high-energy parts of the scaleogram.

In [ ]:
from scipy.signal import find_peaks

def mask_sig(n, peaks, sr=22050, dur=0.1):
    mask = np.zeros(n)
    subm = int(sr*dur*0.5)
    if len(peaks > 0):
        for i in range(len(peaks)):
            mask[max(peaks[i]-subm, 0): min(peaks[i]+subm, n)] = 1
    return mask

fig, axes = plt.subplots(2, 1, figsize=(16,4))
# peak detection + gliding window
peaks, _ = find_peaks(v1, prominence=0.3)
axes[0].plot(v1)
axes[0].plot(peaks, v1[peaks], "x")
m = mask_sig(len(v1), peaks, SR, 0.3)
axes[1].plot(m)
plt.show()

Next, we extract the mask segments with other Scipy functions:

In [ ]:
def get_mask(vdata, prom=0.2, dur=0.2, sr=22050):
    peaks, _ = find_peaks(vdata, prominence=prom)
    return mask_sig(len(vdata), peaks, sr, dur)
    
def get_regions(mask, sr, species, filename):
    regions = scipy.ndimage.find_objects(scipy.ndimage.label(mask)[0])
    regs = []
    for r in regions:
        dur = round((r[0].stop-r[0].start)/sr,3)
        regs.append([r[0].start, r[0].stop, dur,species,filename])
    return pd.DataFrame(regs, columns=['Start', 'End', 'Duration','Species','File'])

In [ ]:
mask = get_mask(v1, prom=0.3, dur=0.2, sr=SR)
df = get_regions(mask, SR, 'btbwar', 'XC139608.mp3')
df.head(6)

We are only interested in segments of a certain length, say 1s or more:

In [ ]:
df = df[df.Duration >= 1.0]
df = df.reset_index(drop=True)
df

Plot those regions to check that we captured the signatures:

In [ ]:
# we plot two seconds
plot_sigx2(d1[df.Start[0]:df.Start[0]+2*SR], d1[df.Start[1]:df.Start[1]+2*SR], 
           name1='btbwar 1', name2='btbwar 2')

Not bad at all! Let's try the other file:

In [ ]:
mask = get_mask(v2, prom=0.3, dur=0.3, sr=SR)
df = get_regions(mask, SR, 'btnwar', 'XC135117.mp3')
df = df[df.Duration >= 1.0]
df = df.reset_index(drop=True)
df

In [ ]:
plot_sigx2(d2[df.Start[0]:df.Start[0]+2*SR], d2[df.Start[1]:df.Start[1]+2*SR], 
           name1='btnwar 1', name2='btnwar 2')

The second signature is clipped a bit, but that is OK. All five signatures were detected. The final step is to combine everything into a function that will extract signatures from a file and dump them as jpeg images.

A rule-based signature extraction algorithm is pretty much doomed on complex or noisy signals. A better solution is to train a binary classifier to extract signatures on a reasonable scale (1-5s), and then feed the extracted signatures to a species classifier. Breaking this problem down into two steps simplifies things a lot, and each step can be optimized independently. 

In [ ]:
# free up memory
%xdel d1
%xdel d2
%xdel cs1
%xdel cs2

# Create images

In [ ]:
def img_resize(cs, w=512, h=512, log=True, lthres=-30):
    buf = io.BytesIO()
    if log == True:
        plt.imsave(buf, 20*np.log10(np.abs(cs)), cmap=CMAP, format='png', vmax=0, vmin=lthres)
    else:
        plt.imsave(buf, np.abs(cs), cmap=CMAP, format='png')
    buf.seek(0)
    img_bytes = np.asarray(bytearray(buf.read()), dtype=np.uint8)
    img = cv2.imdecode(img_bytes, cv2.IMREAD_COLOR)
    return cv2.resize(img, (w, h), interpolation=cv2.INTER_NEAREST)

# Parameters:
#   filename: mp3-file
#   voices: # of scales per octave
#   sr: sampling frequency (Hz)
#   low_freq: low freq cutoff (Hz)
#   thres: scaleogram threshold (dB)
#   prom: peak detect prominence
#   peakdur: peak extension (s)
#   sigthres: smallest signature detection to process (s)
#   siglen: length of output signature (s)
#   img_size: output image size
#   outdir: output directory
def scaleo_extract(filename, voices=12, sr=22050, low_freq=40, thres=-30, prom=0.3, 
                   peakdur=0.3, sigthres=1, siglen=2, img_size=512, outdir='.'):
    d = rd_file(filename)
    cs, _ = cwt2(d, nv=voices, sr=sr, low_freq=low_freq) # wavelet transform
    v = calc_var(cs, thres) # coefficient variance
    peaks, _ = find_peaks(v, prominence=prom) 
    m = mask_sig(len(v), peaks, sr=sr, dur=peakdur) # create signal mask
    df = get_regions(m, sr, filename.split('/')[-2], filename.split('/')[-1])
    df = df[df.Duration >= sigthres] # filter out insignificant signatures
    df = df.reset_index(drop=True)
    if len(df) > 0:
        for i in range(len(df)):
            img = img_resize(cs[:,df.Start[i]:df.Start[i]+siglen*sr], 
                             w=img_size, h=img_size, log=True, lthres=thres)
            fn = df.Species[i]+'-'+filename.split('/')[-1].split('.')[-2]+"-{:03d}.jpg".format(i) 
            cv2.imwrite(outdir+'/'+fn, img)
    return df

Process a few files and see what we get.

In [ ]:
%mkdir ./tmp

In [ ]:
flist = ['/btbwar/XC139608.mp3', '/btbwar/XC51863.mp3', '/btbwar/XC134502.mp3', '/btbwar/XC415596.mp3']
for i in flist:
    scaleo_extract(TRAIN_DIR+i, outdir='./tmp')

In [ ]:
import glob

images = glob.glob('./tmp' + "/btbwar*.jpg")
plt.figure(figsize=(20,20))
columns = 4
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(mpimg.imread(image))
    plt.axis('off')

Now, another warbler.

In [ ]:
flist = ['/btnwar/XC135117.mp3', '/btnwar/XC135495.mp3', '/btnwar/XC173264.mp3', '/btnwar/XC501261.mp3', '/btnwar/XC486860.mp3']
for i in flist:
    scaleo_extract(TRAIN_DIR+i, outdir='./tmp')

In [ ]:
images = glob.glob('./tmp' + "/btnwar*.jpg")
plt.figure(figsize=(20,25))
columns = 5
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(mpimg.imread(image))
    plt.axis('off')

We extracted 4 non-signatures from file XC501261 - should not be a problem to do a post check to detect and remove false signatures. Or better: Train an ML model (binary classifier) to detect signatures.

# Color or grayscale?
The scaleograms represent magnitude of the wavelet coefficients. Pseudo-coloring is used here for human perception. But there is really not more information here than in a grayscale image. Which colormap to use for training? Well, if we think of augmentation with brightness or contrast adjustments, those are only "correct" for grayscale. On the other hand, image models might train better with color inputs... So maybe we need to try both - or even an ensemble of the two!

# Summary
This notebook shows how to use the contiunous wavelet transform to create scaleograms from non-stationary audio signals, with the purpose of using an image classifier to detect bird species. The outline of a two-stage ML pipeline is starting to emerge:  
* Step 1: Train a binary classifier to detect and extract bird songs on a reasonable scale (1-5s)
* Step 2: Feed the bird song segments to a species classifier  

All using the wavelet transform and scaleograms of course.

In [ ]:
!rm -fr tmp